# This Code Converts Kialo's Data from .txt format to Tabular Format in a DataFrame

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install ipython-autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.6 MB/s eta 0:00:00


In [5]:
%load_ext autotime
import pandas as pd
import numpy as np
# import pickle5 as pickle
import re
from tqdm.notebook import tqdm
tqdm.pandas()
import os
import pickle as pkl

time: 339 ms (started: 2023-05-19 19:39:39 +00:00)


In [3]:
data_dir = '.../Datasets/Kialo-Discussions/'

In [6]:
pattern = r".*[^\d]+\.txt$"
for filename in os.listdir(data_dir):
    if re.match(pattern, filename):
        print(filename)
        # file_path = os.path.join(data_dir, filename)
        # os.remove(file_path)

time: 16 s (started: 2023-05-19 19:39:41 +00:00)


In [7]:
# def get_references

def parse_row(row_x):
  # row_x = row_x
  index_x = re.findall(r"^[\d\.]+", row_x)[0]
  row_x = re.sub(r"^[\d\.]+ ", '', row_x)
  row_x = re.sub(":\n", ': ', row_x)
  if (len(re.findall(r"^(Con): ", row_x))>=1) or (len(re.findall(r"^(Pro): ", row_x))>=1):
    stance = row_x.split(': ')[0]
    row_x = re.sub(r"^((Pro)|(Con)): ", '', row_x)
  else:
    if index_x == '1.':
      stance = 'Topic'
    elif len(index_x.split('.'))==3:
      stance = 'Thesis'
    else:
      stance = 'Error'
    row_x = re.sub(r"^((Topic)|(Thesis)): ", '', row_x)
  row_x = re.sub(r"(\n)$", '', row_x)
  row_x = row_x.strip()
  return pd.Series([row_x,index_x,stance])


time: 834 µs (started: 2023-05-19 19:40:04 +00:00)


In [8]:
def get_overall_stance(stance_x,index_x,df):
  stance_x = re.sub(r"(\n)$", '', stance_x)
  distance = 0
  if stance_x == 'Topic' or stance_x == 'Thesis':
    return pd.Series([stance_x,distance])
  while (stance_x == 'Topic' or stance_x == 'Thesis') == False:
    distance = distance + 1
    index_x = re.sub(r"\d+\.$", '', index_x)
    stance_new = df.loc[df['text_index'] == index_x, 'stance'].values[0]
    if (stance_new == 'Topic' or stance_new == 'Thesis') == True:
      return pd.Series([stance_x,distance])
      break
    if stance_new == 'Con':
      if stance_x == 'Pro':
        stance_x = 'Con'
      elif stance_x == 'Con':
        stance_x = 'Pro'

time: 863 µs (started: 2023-05-19 19:40:06 +00:00)


In [9]:
def merge_enter_lines(list_x):
  list_x = list(filter(lambda a: a != '\n', list_x))
  output  = []
  for i,row_x in enumerate(list_x):
      row_x = row_x.strip()
      if len(re.findall(r"^\d\.[\d\.]* ", row_x))>0:
          output.append(row_x)
      # elif (i<=5) and (len(re.findall(r"^\d\.[\d\.]*", row_x))>0):
          # output.append(row_x)
      else:
          output[-1] += "\n " + row_x
  return output

def source_list2dict(sources):
  sources_dict = {}
  for row_x in sources:
    row_x = re.sub(r"(\n)$", '', row_x)
    if len(row_x.split(' '))>1:
      index, link = row_x.split(' ')[0], row_x.split(' ')[1]
      if re.match('\[\d+\]',index) and re.match('http\S+|www.\S+',link):
        sources_dict[index] = link
  return sources_dict

def insert_links(row_x, source):
  row_x = re.sub("\[\d+\]", lambda m: source.get(m.group()), row_x)
  return row_x

def insert_sees(row_x, source):
  if re.findall("-> [A-Za-z]+ [\d\.]+", row_x):
    row_x = re.sub("-> [A-Za-z]+ ", '', row_x)
    row_x = re.sub("[\d\.]+", lambda m: source.get(m.group()), row_x)
  return row_x

time: 3.67 ms (started: 2023-05-19 19:40:07 +00:00)


In [ ]:
all_Kialo_claims_df_meta_data = pd.read_hdf(my_dir+'Kialo_MetaData_popular.hdf')#[['post_id','language','text_vectorized','all-mpnet-base-v2']]
language_dict = dict(zip(all_Kialo_claims_df_meta_data['post_id'],all_Kialo_claims_df_meta_data['language']))
def text2DF(text_file_path):
  print('\n')
  print(text_file_path)
  f = open(text_file_path)
  all_text = f.readlines()
  Discussion_Name = re.sub('^Discussion Name: ', '', all_text[0])
  Discussion_Name = re.sub('^Discussion Title: ', '', Discussion_Name)
  Discussion_Name = re.sub('\n$', '', Discussion_Name)
  print(Discussion_Name)
  try:
    pros_cons = all_text[2:all_text[2:].index('Sources:\n')+1]
    sources = all_text[all_text[2:].index('Sources:\n')+3:]
    sources = source_list2dict(sources)
  except:
    pros_cons = all_text[2:]
    sources = {}
  pros_cons = merge_enter_lines(pros_cons)
  data_i = pd.DataFrame()
  discussion_id = re.sub('\.txt$', '', text_file_path).split('-')[-1]
  data_i['text_unclean'] = pros_cons
  data_i[['text', 'text_index','stance']]=data_i['text_unclean'].apply(lambda x: parse_row(x))
  data_i['text']=data_i['text'].apply(lambda x: insert_links(row_x=x, source=sources))
  data_i['text']=data_i['text'].apply(lambda x: insert_sees(row_x=x, source=dict(zip(data_i['text_index'],data_i['text']))))
  data_i['post_id'] = int(discussion_id)
  data_i['discussion_name'] = Discussion_Name
  data_i['file_name'] = text_file_path.split('/')[-1]
  num_errors = len(data_i[data_i['stance']=='Error'])
  print(f'number of errors: {num_errors}')
  if num_errors == 0:
    data_i[['stance_to_root','distance_to_root']] = data_i.apply(lambda x: get_overall_stance(stance_x=x['stance'],index_x=x['text_index'],df=data_i),axis=1)
  if num_errors > 0:
    data_i['stance_to_root'] = 'Error'
    data_i['distance_to_root'] = 'Error'

  return data_i

all_files = os.listdir(data_dir)
data_all = pd.DataFrame()
for file_x in tqdm(all_files):
  post_id = int(file_x.split('-')[-1].split('.')[0].split(' ')[0])
  if post_id in language_dict:
    if language_dict[post_id] == 'en':
      text_file_path = data_dir + file_x
      data_all = pd.concat([data_all,text2DF(text_file_path)],ignore_index=True)
data_all

In [ ]:
data_all.to_csv('.../Datasets/Kialo-Discussions/kialo_popular_tabular_format', index=None)

time: 8.89 s (started: 2023-03-02 14:51:06 +00:00)
